[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/okada-tak/deep-learning-from-scratch-2/blob/master/notebooks/ch05.ipynb)

# 5章 リカレントニューラルネットワーク(RNN) のまとめ
- RNNはループする経路があり、それによって「隠れ状態」を内部に記憶することができる
- RNNのループ経路を展開することで、複数のRNNレイヤがつながったネットワークと解釈することができ、通常の誤差逆伝播法によって学習することができる（＝BPTT）
- 長い時系列データを学習する場合は、適当な長さでデータのまとまりを作り－これを「ブロック」という－、ブロック単位でBPTTによる学習を行う（＝Truncated BPTT）
- Truncated BPTTでは逆伝播のつながりのみを切断する
- Truncated BPTTでは順伝播のつながりは維持するため、データは”シーケンシャル”に与える必要がある
- 言語モデルは、単語の羅列を確率として解釈する
- RNNレイヤを利用した条件付き言語モデルは、（理論的には）それまで登場した単語の情報を記憶することができる

単純なフィードフォワード・ネットワークでは、時系列データの性質（パターン）を十分に学習することができない

## 5.1 確率と言語モデル

### 5.1.1 word2vecを確率の視点から眺める
$w_{t-1}$と$w_{t+1}$が与えられたとき、ターゲットが$w_t$となる確率  
$$
P(w_t|w_{t-1}, w_{t+1})
$$

左側2つの単語だけをコンテキストとして考える
$$
P(w_t|w_{t-2}, w_{t-1})
$$

CBOWモデルが扱う損失関数
$$
L=-logP(w_t|w_{t-2}, w_{t-1})
$$

### 5.1.2 言語モデル
$w_1$,…,$w_m$の$m$個の単語からなる文章  
$w_1$,…,$w_m$という順序で単語が出現する確率  
$$
\begin{split}
P(w_1,…,w_m)&=P(w_m|w_1,…,w_{m-1})P(w_{m-1}|w_1,…,w_{m-2})...P(w_3|w_1,w_2)P(w_2|w_1)p(w_1)\\  
&=\prod_{t=1}^mP(w_t|w_1,…,w_{t-1})
\end{split}
$$
目標：  
$P(w_t|w_1,…,w_{t-1})$を求めること。これがわかれば言語モデルの同時確率$P(w_1,…,w_m)$を求められる

### 5.1.3 CBOWモデルを言語モデルに？
コンテキストのサイズを限定することでCBOWモデルを言語モデルに適用
$$
P(w_1,…,w_m)=\prod_{t=1}^mP(w_t|w_1,…,w_{t-1})≈\prod_{t=1}^mP(w_t|w_{t-2},w_{t-1})
$$
直前の2つの単語だけに依存するとして2階のマルコフ連鎖  
CBOWモデルではコンテキスト内の単語の並びが無視されるのが問題


## 5.2 RNNとは

### 5.2.1 循環するニューラルネットワーク
### 5.2.2 ループの展開
$$
\mathbf h_t=tanh(\mathbf h_{t-1}\mathbf W_\mathbf h+\mathbf x_t\mathbf W_\mathbf x+\mathbf b)
$$

### 5.2.3 Backpropagation Through Time
長い時系列データを扱うと消費する計算リソースが増大する  
逆伝播時の勾配が不安定になる

### 5.2.4 Truncated BPTT
ネットワークの逆伝播のつながりだけを適当な長さで切り取り、切り取られたネットワーク単位で学習する

順伝播のつながりは切断しないのでデータをシーケンシャルに与える必要がある

### 5.2.5 Truncated BPTTのミニバッチ学習
データの与え方：

*   データをシーケンシャルに与えること
*   各バッチでデータを与える開始位置をずらすこと



## 5.3 RNNの実装
RNNの1ステップの処理を行うクラスをRNNクラス

そのRNNクラスを利用して、Tステップの処理をまとめて行うレイヤをTimeRNNクラス

### 5.3.1 RNNレイヤの実装
RNNの順伝播  
$$
\mathbf h_t=tanh(\mathbf h_{t-1}\mathbf W_\mathbf h+\mathbf x_t\mathbf W_\mathbf x+\mathbf b)
$$

## ■Colaboratory用
Google Colaboratoryの場合、Google Driveに  
dl-from-scratch-2/ch05  
というフォルダを用意し、そこにこのjupyter notebookを配置。  
(dl-from-scratch-2の部分は任意。)  
また、datasetフォルダとcommonフォルダを
dl-from-scratch-2/dataset  
dl-from-scratch-2/common
にコピーしておく。  

以下のセルでGoogle Driveをマウント。許可を求められるので許可する。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## ■Colaboratory用
chdirする。

In [ ]:
import sys,os
os.chdir('/content/drive/My Drive/dl-from-scratch-2/ch05')
os.getcwd()

common/time_layers.py

In [ ]:
from common.np import *  # import numpy as np (or import cupy as np)
from common.layers import *
from common.functions import softmax, sigmoid

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, Wh.T)
        dWx = np.dot(x.T, dt)
        dx = np.dot(dt, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

### 5.3.2 Time RNNレイヤの実装


common/time_layers.py

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

## 5.4 時系列データを扱うレイヤの実装

### 5.4.1 RNNLMの全体図